In [1]:
import awswrangler as wr
import pandas as pd
import boto3
import json

In [2]:
boto3.setup_default_session(profile_name='geekfox-pawprint')

In [28]:
df = wr.s3.read_json('s3://bucket/taxonomy/taxonomy/taxonomy_full.json', lines=True)

In [38]:
import io 

s3 = boto3.resource('s3')
obj = s3.Object('bucket', 'taxonomy/taxonomy/taxonomy_full.json')
body = obj.get()['Body'].read()
df = pd.read_json(io.StringIO(body.decode('utf-8')))

In [34]:
df.head()

,taxonomy_nodes
0,"{'query': ['301158'], 'taxonomy': {'tax_id': 3..."
1,"{'query': ['374419'], 'taxonomy': {'tax_id': 3..."
2,"{'query': ['1496132'], 'taxonomy': {'tax_id': ..."
3,"{'query': ['1296583'], 'taxonomy': {'tax_id': ..."
4,"{'query': ['1209081'], 'taxonomy': {'tax_id': ..."


In [39]:

df = pd.json_normalize(df['taxonomy_nodes'].tolist())

In [40]:
df.head()

,query,taxonomy.tax_id,taxonomy.organism_name,taxonomy.common_name,taxonomy.lineage,taxonomy.rank,taxonomy.has_described_species_name,taxonomy.children,taxonomy.descendent_with_described_species_names_count,taxonomy.counts
0,[301158],301158,Haplonycteris fischeri,Philippine pygmy fruit bat,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",SPECIES,True,NaN,NaN,NaN
1,[374419],374419,Megaerops kusnotoi,Javan tailless fruit bat,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",SPECIES,True,NaN,NaN,NaN
2,[1496132],1496132,Pteropus tokudae,Guam flying fox,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",SPECIES,True,NaN,NaN,NaN
3,[1296583],1296583,Vespertilionidae sp. BOLD:AAC0445,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",SPECIES,NaN,NaN,NaN,NaN
4,[1209081],1209081,Myotis planiceps,flat-headed Myotis,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",SPECIES,True,NaN,NaN,NaN


In [7]:
df[~df['taxonomy.rank'].isin(['SPECIES', 'SUBSPECIES'])].head()

,query,taxonomy.tax_id,taxonomy.organism_name,taxonomy.common_name,taxonomy.lineage,taxonomy.children,taxonomy.descendent_with_described_species_names_count,taxonomy.rank,taxonomy.has_described_species_name,taxonomy.counts


In [8]:
df[~df['taxonomy.children'].isna()].head()

,query,taxonomy.tax_id,taxonomy.organism_name,taxonomy.common_name,taxonomy.lineage,taxonomy.children,taxonomy.descendent_with_described_species_names_count,taxonomy.rank,taxonomy.has_described_species_name,taxonomy.counts
0,[59472],59472,Pipistrellus kuhlii,Kuhl's pipistrelle,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...","[1707946, 983528]",2.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 29200}, ..."
11,[159332],159332,Myotis oxyotus,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[1908665],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."
26,[384634],384634,Myotis chiloensis,Chilean myotis,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[2977340],1.0,SPECIES,True,NaN
30,[242915],242915,Plecotus macrobullaris,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[272800],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."
46,[392318],392318,Myotis bombinus,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[392319],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."


In [9]:
df[df['taxonomy.tax_id'] == 1908665].head()

,query,taxonomy.tax_id,taxonomy.organism_name,taxonomy.common_name,taxonomy.lineage,taxonomy.children,taxonomy.descendent_with_described_species_names_count,taxonomy.rank,taxonomy.has_described_species_name,taxonomy.counts
380,[1908665],1908665,Myotis oxyotus gardneri,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",NaN,NaN,SUBSPECIES,True,NaN


In [27]:
df[~df['taxonomy.descendent_with_described_species_names_count'].isna()].head()

,query,taxonomy.tax_id,taxonomy.organism_name,taxonomy.common_name,taxonomy.lineage,taxonomy.children,taxonomy.descendent_with_described_species_names_count,taxonomy.rank,taxonomy.has_described_species_name,taxonomy.counts
0,[59472],59472,Pipistrellus kuhlii,Kuhl's pipistrelle,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...","[1707946, 983528]",2.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 29200}, ..."
11,[159332],159332,Myotis oxyotus,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[1908665],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."
26,[384634],384634,Myotis chiloensis,Chilean myotis,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[2977340],1.0,SPECIES,True,NaN
30,[242915],242915,Plecotus macrobullaris,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[272800],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."
46,[392318],392318,Myotis bombinus,NaN,"[1, 131567, 2759, 33154, 33208, 6072, 33213, 3...",[392319],1.0,SPECIES,True,"[{'type': 'COUNT_TYPE_GENE', 'count': 37}, {'t..."


In [15]:
df.columns

Index(['query', 'taxonomy.tax_id', 'taxonomy.organism_name',
       'taxonomy.common_name', 'taxonomy.lineage', 'taxonomy.children',
       'taxonomy.descendent_with_described_species_names_count',
       'taxonomy.rank', 'taxonomy.has_described_species_name',
       'taxonomy.counts'],
      dtype='object')

In [9]:
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig, DatasourceConfig, AnonymizedUsageStatisticsConfig
from great_expectations.dataset import PandasDataset
import great_expectations as gx

In [10]:
context = gx.get_context(
    DataContextConfig(
        datasources={
            "pandas_datasource": DatasourceConfig(
                class_name="PandasDatasource"                
            )
        },
        stores={
            "expectations_store": {
                "class_name": "ExpectationsStore",
                "store_backend": {
                    "class_name": "TupleFilesystemStoreBackend",
                    "base_directory": "/Volumes/GeekFoxLab/Projetos/Pawprint/config/expectations",
                }
            }
        },
        expectations_store_name="expectations_store",
    )
)

In [11]:
suite = context.get_expectation_suite('taxonomy.taxonomy')

In [12]:
batch = PandasDataset(df, expectation_suite=suite)
results = batch.validate()
results

{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "expectation_type": "expect_table_columns_to_match_ordered_list",
        "kwargs": {
          "column_list": [
            "query",
            "taxonomy.tax_id",
            "taxonomy.organism_name",
            "taxonomy.common_name",
            "taxonomy.lineage",
            "taxonomy.children",
            "taxonomy.descendent_with_described_species_names_count",
            "taxonomy.rank",
            "taxonomy.has_described_species_name",
            "taxonomy.counts"
          ]
        },
        "meta": {}
      },
      "result": {
        "observed_value": [
          "query",
          "taxonomy.tax_id",
          "taxonomy.organism_name",
          "taxonomy.lineage",
          "taxonomy.rank",
          "taxonomy.has_described_species_name",
          "taxonomy.common_name",
          "taxonomy.children",
          "taxonomy.descendent_with_described_species_names